In [36]:
import json
import glob
import matplotlib.pyplot as plt

# データセット作成
## 形式
- (pokemon_id, moves_id, label, method)

In [38]:
moves = glob.glob('../data/moves/*.json')
move_dict = {}
remove_move = set()
CANT_USE_MOVE_KEY = "This move can\u2019t be used.\nIt\u2019s recommended that this move is forgotten.\nOnce forgotten, this move can\u2019t be remembered."
# {
#     "flavor_text": "This move can\u2019t be used.\nIt\u2019s recommended that this move is forgotten.\nOnce forgotten, this move can\u2019t be remembered.",
#     "language": {
#         "name": "en",
#         "url": "https://pokeapi.co/api/v2/language/9/"
#     },
#     "version_group": {
#         "name": "sword-shield",
#         "url": "https://pokeapi.co/api/v2/version-group/20/"
#     }
# },
for pp in moves:
    with open(pp, 'r') as f:
        print(pp)
        move = json.load(f)
    move_id = move["id"]
    if move["damage_class"]["name"] == "status":
        remove_move.add(move_id)
        continue
    if len(move["flavor_text_entries"]) == 0:
        remove_move.add(move_id)
        continue
    if move["pp"] <= 1:
        # z技
        remove_move.add(move_id)
        continue
    if move["name"].startswith("max-"):
        remove_move.add(move_id)
        continue
    descriptions_in_en = list(filter(lambda x: x["language"]["name"] == 'en' and x["flavor_text"] != CANT_USE_MOVE_KEY, move["flavor_text_entries"]))
    if len(descriptions_in_en) == 0:
        remove_move.add(move_id)
        continue
    move_name = move["name"]
    move_dict[move_name] = move_id
    

../data/moves\1.json
../data/moves\10.json
../data/moves\100.json
../data/moves\101.json
../data/moves\102.json
../data/moves\103.json
../data/moves\104.json
../data/moves\105.json
../data/moves\106.json
../data/moves\107.json
../data/moves\108.json
../data/moves\109.json
../data/moves\11.json
../data/moves\110.json
../data/moves\111.json
../data/moves\112.json
../data/moves\113.json
../data/moves\114.json
../data/moves\115.json
../data/moves\116.json
../data/moves\117.json
../data/moves\118.json
../data/moves\119.json
../data/moves\12.json
../data/moves\120.json
../data/moves\121.json
../data/moves\122.json
../data/moves\123.json
../data/moves\124.json
../data/moves\125.json
../data/moves\126.json
../data/moves\127.json
../data/moves\128.json
../data/moves\129.json
../data/moves\13.json
../data/moves\130.json
../data/moves\131.json
../data/moves\132.json
../data/moves\133.json
../data/moves\134.json
../data/moves\135.json
../data/moves\136.json
../data/moves\137.json
../data/moves\138

In [39]:
pokemons = glob.glob('../data/pokemons/*.json')
pokemon_move_mapping = {}

for pp in pokemons:
    with open(pp, 'r') as f:
        pokemon = json.load(f)
    pokemon_id = pokemon["id"]
    moves = pokemon["moves"]
    pokemon_moves = {}
    for move in moves:
        move_name = move["move"]["name"]
        last_varsion = move["version_group_details"][-1]
        learn_method = last_varsion["move_learn_method"]["name"]
        if move_name not in move_dict:
            continue
        move_id = move_dict[move_name]
        pokemon_moves[move_id] = learn_method
    pokemon_move_mapping[pokemon_id] = pokemon_moves

In [40]:
text = ""
count = 0
max_length = 1010 * 900
for pokemon_id in range(1,1011):
    pokemon = pokemon_move_mapping[pokemon_id]
    for move_id in range(1, 901):
        if move_id in remove_move:
            continue
        if move_id in pokemon:
            # この技をこのポケモンが覚える
            text += f'{pokemon_id}, {move_id}, 1, {pokemon[move_id]}\n'
        else:
            # この技をこのポケモンが覚えない
            text +=  f'{pokemon_id}, {move_id}, 0,\n'
        count += 1
        if count % 10000 == 0:
            print(f'\r{count * 100 / max_length}%', end='')

55.005500550055004%

In [41]:
with open('D:/tanaka/Documents/poke-move/data/my-dataset/dataset_without_status_z_no_leadned.txt', 'w') as f:
    f.write(text)

In [42]:
with open('D:/tanaka/Documents/poke-move/data/my-dataset/remove_move_ids', 'w') as f:
    f.write("\n".join(list(map(str, list(remove_move)))))

In [43]:
# remove_moveの名前確認

import sys
import os

sys.path.append(os.path.abspath("../"))
from learning.Entity.CustomDataset import PokemonMoveDataset
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")  # GPUデバイスを取得
else:
    device = torch.device("cpu")  # CPUデバイスを取得
dataset = PokemonMoveDataset('D:/tanaka/Documents/poke-move/data/my-dataset/dataset.txt', device)

In [44]:
import json
text = ""
for move_id in list(remove_move):
    move = dataset.moves[move_id-1]
    with open(f'D:/tanaka/Documents/poke-move/data\moves\{move_id}.json') as f:
        m = json.load(f)
    name = list(filter(lambda x: x["language"]["name"] == 'ja', m["names"]))
    if len(name) == 0:
        print(move_id)
        continue
    name = name[0]
    text += name["name"] + '\n'


875
876


In [45]:
with open('remove_move_names.txt', 'w') as f:
    f.write(text)